<a href="https://colab.research.google.com/github/averma12/Kaggle/blob/master/NYC_taxi_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! pip install kaggle

In [0]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

In [0]:
auth.authenticate_user()

In [7]:
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [0]:
os.rmdir('/content/.kaggle/kaggle.json')

In [0]:
os.makedirs('/content/.kaggle/',exist_ok=True)

In [0]:
os.makedirs('/root/.kaggle/',exist_ok=True)

In [0]:
import json
token = {"username":"averma12","key":"f345e377d75ac0a4fc7f21f0e26c0aaf"}

In [0]:
with open('/content/.kaggle/kaggle.json', 'w') as file:
  json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [18]:
!kaggle competitions download -c new-york-city-taxi-fare-prediction -p /content/kaggle

  0% 0.00/335k [00:00<?, ?B/s]
100% 335k/335k [00:00<00:00, 51.8MB/s]
  0% 0.00/960k [00:00<?, ?B/s]
100% 960k/960k [00:00<00:00, 61.9MB/s]
 99% 1.55G/1.56G [00:24<00:00, 85.1MB/s]
100% 1.56G/1.56G [00:24<00:00, 68.8MB/s]
  0% 0.00/486 [00:00<?, ?B/s]
100% 486/486 [00:00<00:00, 463kB/s]


In [19]:
! unzip /content/kaggle/train.csv.zip

Archive:  /content/kaggle/train.csv.zip
  inflating: train.csv               


In [0]:
import numpy as np
import pandas as pd

In [14]:
df_10000 = pd.read_csv('train.csv', nrows=1000)
df_10000.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [0]:
#infer datatypes
#df_10000.dtypes
cols = ['fare_amount', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count']

In [0]:
types = {
    'fare_amount': 'float16',
    'pickup_datetime': 'str',
    'pickup_longitude': 'float16',
    'pickup_latitude': 'float16',
    'dropoff_longitude': 'float16',
    'dropoff_latitude': 'float16',
    'passenger_count': 'uint8'
}

In [5]:
%time df_all = pd.read_csv('train.csv',usecols=cols,dtype=types)

CPU times: user 1min 42s, sys: 7.82 s, total: 1min 50s
Wall time: 3min 46s


In [8]:
df_all.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,5.542386e+07,5.542386e+07,5.542386e+07,5.542348e+07,5.542348e+07,5.542386e+07
mean,NaN,NaN,NaN,NaN,NaN,1.685380e+00
std,NaN,NaN,NaN,NaN,NaN,1.327664e+00
min,-3.000000e+02,-3.442000e+03,-3.492000e+03,-3.442000e+03,-3.548000e+03,0.000000e+00
25%,6.000000e+00,-7.400000e+01,4.075000e+01,-7.400000e+01,4.071875e+01,1.000000e+00
50%,8.500000e+00,-7.400000e+01,4.075000e+01,-7.400000e+01,4.075000e+01,1.000000e+00
75%,1.250000e+01,-7.393750e+01,4.078125e+01,-7.393750e+01,4.078125e+01,2.000000e+00
max,inf,3.458000e+03,3.408000e+03,3.458000e+03,3.538000e+03,2.080000e+02


In [6]:
df_all.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.500000,2009-06-15 17:26:21 UTC,-73.8750,40.71875,-73.8125,40.71875,1
1,16.906250,2010-01-05 16:52:16 UTC,-74.0000,40.71875,-74.0000,40.78125,1
2,5.699219,2011-08-18 00:35:00 UTC,-74.0000,40.75000,-74.0000,40.75000,2
3,7.699219,2012-04-21 04:30:42 UTC,-74.0000,40.71875,-74.0000,40.75000,1
4,5.300781,2010-03-09 07:51:00 UTC,-73.9375,40.78125,-73.9375,40.78125,1


**Feature Engineering to Convert Categorical columns to numbers to parse dates to acceptable value etc
**

In [7]:
%time df_all['pickup_datetime'] = df_all['pickup_datetime'].str.slice(0, 16)

CPU times: user 17.3 s, sys: 2.34 s, total: 19.6 s
Wall time: 19.6 s


In [8]:
%time df_all['pickup_datetime'] = pd.to_datetime(df_all['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')

CPU times: user 13.3 s, sys: 371 ms, total: 13.7 s
Wall time: 13.6 s


In [10]:
df_all.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.500000,2009-06-15 17:26:00+00:00,-73.8750,40.71875,-73.8125,40.71875,1
1,16.906250,2010-01-05 16:52:00+00:00,-74.0000,40.71875,-74.0000,40.78125,1
2,5.699219,2011-08-18 00:35:00+00:00,-74.0000,40.75000,-74.0000,40.75000,2
3,7.699219,2012-04-21 04:30:00+00:00,-74.0000,40.71875,-74.0000,40.75000,1
4,5.300781,2010-03-09 07:51:00+00:00,-73.9375,40.78125,-73.9375,40.78125,1


In [0]:
#will do later
def add_datepart(dataframe,column,drop=True):
  field = dataframe[column]
  for n in ('Year','Month','Day','Dayofweek','Dayofyear','hour'):
   
  

In [0]:
#extract month
df_all['pickup_datetime_month'] = df_all['pickup_datetime'].dt.month
#extract year
df_all['pickup_datetime_year'] = pd.DatetimeIndex(df_all['pickup_datetime']).year
#extract day
df_all['pickup_datetime_day'] = df_all['pickup_datetime'].dt.day
#extract day name
df_all['pickup_datetime_weekday_name'] = df_all["pickup_datetime"].dt.weekday_name
#extract hour
df_all['pickup_datetime_hour'] = df_all["pickup_datetime"].dt.hour

In [13]:
df_all.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_month,pickup_datetime_year,pickup_datetime_day,pickup_datetime_weekday_name,pickup_datetime_hour
0,4.500000,2009-06-15 17:26:00+00:00,-73.8750,40.71875,-73.8125,40.71875,1,6,2009,15,Monday,17
1,16.906250,2010-01-05 16:52:00+00:00,-74.0000,40.71875,-74.0000,40.78125,1,1,2010,5,Tuesday,16
2,5.699219,2011-08-18 00:35:00+00:00,-74.0000,40.75000,-74.0000,40.75000,2,8,2011,18,Thursday,0
3,7.699219,2012-04-21 04:30:00+00:00,-74.0000,40.71875,-74.0000,40.75000,1,4,2012,21,Saturday,4
4,5.300781,2010-03-09 07:51:00+00:00,-73.9375,40.78125,-73.9375,40.78125,1,3,2010,9,Tuesday,7


In [0]:
df_10000.pickup_datetime = df_10000['pickup_datetime'].str.slice(0, 16)
df_10000.pickup_datetime = pd.to_datetime(df_10000['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')

In [20]:
getattr(df_10000.pickup_datetime.dt,'year')

0      2009
1      2010
2      2011
3      2012
4      2010
5      2011
6      2012
7      2012
8      2012
9      2009
10     2012
11     2012
12     2009
13     2013
14     2011
15     2013
16     2014
17     2009
18     2010
19     2014
20     2010
21     2013
22     2009
23     2010
24     2009
25     2011
26     2011
27     2011
28     2012
29     2013
       ... 
970    2013
971    2010
972    2011
973    2014
974    2015
975    2015
976    2011
977    2012
978    2015
979    2013
980    2011
981    2011
982    2014
983    2009
984    2012
985    2012
986    2010
987    2012
988    2015
989    2013
990    2015
991    2011
992    2014
993    2015
994    2009
995    2011
996    2011
997    2011
998    2014
999    2012
Name: pickup_datetime, Length: 1000, dtype: int64

In [0]:
import dask.dataframe as dd